In [ ]:
"""

Contents:
-data intialization from csv file of words &
 build data structure to house info moving forward
-write to .json formats
-read from existing .json formats
-get list of words to find
-scrape BBC
-scrape voiceTV
"""

In [1]:
import bs4, requests, sys, codecs, urllib.request, re
from bs4 import SoupStrainer
from bs4.element import Comment
import random
#import pythainlp
from pythainlp.tokenize import word_tokenize, sent_tokenize
import string
import json
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import pprint
import translators as ts

pp = pprint.PrettyPrinter()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)

path = "/Users/elyebliss/Desktop/Vocabulary/language_learning/vocab_dfs/"
source_file = "thai.json"

Using state Washington server backend.


In [2]:
##METHODS

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = bs4.BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)



user_agent = 'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers={'User-Agent':user_agent,}
parser = 'html.parser'

In [3]:
def get_known(unknown_list):
    
    count_got = 0
    known_list = []
    for word in unknown_list:
        decision = str(input(word+"\nKnown =k"))
        if decision =='k':
            known_list.append(word)
            count_got +=1
            print("got "+str(count_got))
        elif decision=='q':
            break
        try:
            print(ts.google(word))
        except:
            print('cant find')
    return known_list

In [4]:
def filter_text(webpage,start=None,stop=None,\
                print_word_lvl=False,percent_threshold=None,\
               return_percent=False):


    try:
        request=urllib.request.Request(webpage,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)

        known_array = []
        unk_array = []
        contents_array = sent_tokenize(contents)

        if (start is not None) and (stop is not None):
            contents_array=contents_array[max(start,0):min(stop,len(contents_array))]

        disallowed_words = set()

        total_words = 0
        unknown_words = 0
        
        lines = []
        unknowns = []
        for line in contents_array:


            tokenized = word_tokenize(line)


            add_line = True
            line_total = 0
            line_unks = 0

            unk_str = ""
            for word in tokenized:
                
                total_words +=1
                line_total +=1

                if bool(re.search('[\u0E00-\u0E7F]+', word, flags=re.UNICODE)) and not ((word in vocab['white_listed']) or\
                                                            (word in vocab['black_listed'])):

                        unk_str += '"'+word+'"'+", "
                        if not percent_threshold:
                            add_line = False
                        disallowed_words.add(word)
                        unknown_words +=1
                        line_unks +=1

            if percent_threshold:
                if line_total>0:
                    if (1-(line_unks/line_total))<percent_threshold:
                        add_line = False
            if add_line:
                known_array.append(line)
                unk_array.append("...")
            else:
                known_array.append("...")
                unk_array.append(line)

            if len(unk_str)>0:
                unk_str = unk_str[0:len(unk_str)-2]

            unknowns.append(unk_str)

        if print_word_lvl and total_words>0:
            print("word-level % known = "+str((1-(unknown_words/total_words))*100))
        if return_percent and total_words>0:
            return (1-(unknown_words/total_words))*100
        return_pd = pd.DataFrame(list(zip(known_array,unk_array,unknowns)))
        return_pd.columns = ["knowns","unknowns","unk_words"]

        

        with open(path+'unknown_thai_list.txt',"w") as outfile:
            outfile.write(str(list(disallowed_words)))

        return return_pd

    except:
        pass

In [50]:
"""read from existing .json formats
uncovered, 
2192
2537
"""
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed'])) 

#get 4k freq word info:
with open(path+'th_freq.csv','r') as infile:
    freq_df = pd.read_csv(infile)
uncovered = []
for word in freq_df.word:
    if word not in vocab['white_listed']:
        uncovered.append(word)
print(len(uncovered))



2192
2537


In [29]:
pages = []
titles = []

##BBC
print("Checking BBC Thai")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
request=urllib.request.Request('https://www.bbc.com/thai/topics/cjgn73g98rqt',None,headers)
resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))


for link in soup.find_all('a', href=True):
    if 'https://www.bbc.com/thai/thailand' in str(link['href']):
        pages.append(str(link['href']))
        request=urllib.request.Request(str(link['href']),None,headers) #The assembled request

        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)
        if bool(re.search("(?<=ยอดนิยม หน้าแรก ประเทศไทย ต่างประเทศ วิทยาศาสตร์ สุขภาพ โควิด-19 วิดีโอ ยอดนิยม).{30}",contents)):
            titles.append(re.findall("(?<=ยอดนิยม หน้าแรก ประเทศไทย ต่างประเทศ วิทยาศาสตร์ สุขภาพ โควิด-19 วิดีโอ ยอดนิยม).{30}",contents)[0].strip())

    elif '/thai/thailand' in str(link['href']):
        pages.append('https://www.bbc.com'+str(link['href']))

##VoiceTV
print("Checking VoiceTV")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
request=urllib.request.Request('https://www.voicetv.co.th/topic/%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%80%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%87',None,headers)
resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

 
for link in soup.find_all('a', href=True):
    str_page = 'https://www.voicetv.co.th'+str(link['href'])
    if '/read/' in str(link['href']) and str_page not in pages:
        pages.append(str_page)
        request=urllib.request.Request(str_page,None,headers) #The assembled request

        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)
        titles.append(re.findall("(?<=  :).{30}",contents)[0].strip())

Checking BBC Thai
Checking VoiceTV


In [30]:
#load prev corpus and add
known_corpus = set()
with open(path+"all_known_thai_lines.txt","r",encoding='utf-8') as infile:
    for line in infile.read().split('\n'):
        known_corpus.add(line)
len(known_corpus)

1164

In [31]:
#Find % of all sentences of news
knowns = []
all_lines = 0

known_percents = []

for webpage in pages:
    
    test_df = filter_text(webpage)
    all_lines += len(list(test_df.knowns))
    
    known_percents.append(filter_text(webpage,return_percent=True))
    
    for item in list(test_df.knowns[test_df.knowns != '...']):
        knowns.append(item)
percent = len(knowns)/float(all_lines)
print("percent all known= "+str(percent))


percent all known= 0.16248348745046234


In [32]:
#sorted(list(set(knowns)),key=len,reverse=True)
sorted(list(set(knowns).difference(known_corpus)),key=len,reverse=True)

['เพราะฉะนั้น ถ้าไม่ลงจากหลังเสือตอนนี้ก็จะไม่สามารถลงได้อย่างสวยงาม ',
 'ก็รู้แล้วว่าเป็นดาวคนละดวง แต่ตรงนี้เหมือนเป็นนายกฯ คนละครึ่ง ',
 'การที่จะเป็นรัฐบาลจะต้องได้หลายพื้นที่มากขึ้น เดินมากขึ้น ',
 'แน่นอนว่า รัฐบาลนี้มีอำนาจรัฐจึงพยายามปิดเสียงประชาชน ',
 'Last update Nov 25, 2022 11:28 )   นายกรัฐมนตรี ',
 'เพื่อให้ทุกอย่างในประเทศไทยเดินไปข้างหน้าให้ได้ ',
 'เพราะตอนนั้นมีเวลาน้อย แต่ตอนนี้ทำงานมาปีครึ่ง ',
 'แม้ว่าสภาฯ จะมีเวลาเหลืออีกไม่กี่เดือนก็ตาม ',
 'นั่นคือสิ่งที่ทำให้จิตใจผมอยู่ถึงวันนี้ได้ ',
 'ถึงไม่ชอบก็ยิ้มไว้ก่อนเดี๋ยวก็คุยกันได้เอง ',
 'ทำให้รู้ว่าโลกใบนี้ไม่ใช่โลกใบเดิมของเรา ',
 'ประชาชนต้องมีส่วนรู้ และเข้าใจมากที่สุด ',
 'ตนค่อนข้างกังวลที่บางครั้งพูดกันไม่ได้ ',
 'เราต้องลดของเราให้ได้ โอกาสจึงจะมาถึง ',
 'ตนทราบถึงปัญหาดีว่าเกิดอะไรขึ้นบ้าง ',
 'วันนี้จึงขอให้ติดตามกันในเรื่องนี้ ',
 'เราจึงต้องช่วยกันว่าอะไรทำได้บ้าง ',
 'การจำกัดสิทธิต้องไม่เลือกปฏิบัติ ',
 'มันพร้อมมาตั้งแต่ 6 เดือนที่แล้ว ',
 'มันจะถูกควบคุมการทำงานจากรัฐบาล ',
 'ส่วนต่อมา นโยบายเศรษฐกิ

In [33]:
for item in list(set(knowns)):
    known_corpus.add(item)

with open(path+"all_known_thai_lines.txt","w",encoding='utf-8') as outfile:
    for line in list(set(known_corpus)):
        outfile.write(line+'\n')
len(known_corpus)

1215

In [34]:
websites = set()

with open(path+'viewed_websites_th.txt',"r") as infile:
    for line in infile.read().split('\n'):
        websites.add(line)
print(len(websites))

18


In [35]:
page_titles = list(zip(range(0,len(known_percents)),known_percents,titles,pages))
# (sorted(list(zip(range(0,len(known_percents)),known_percents)),key = lambda x: x[1],reverse=True))
page_titles = sorted(page_titles,key = lambda x: x[1],reverse=True)
page_titles = [item for item in page_titles if not (item[3] in websites)]
page_titles

[(4,
  89.43364327979712,
  '‘ประยุทธ์’ อวยพรปีใหม่สื่อทำเ',
  'https://www.voicetv.co.th/read/0kUrVxJCm'),
 (3,
  88.23529411764706,
  "'ชัยวุฒิ' ยื่นลาออก ส.ส.พปชร.",
  'https://www.voicetv.co.th/read/zf066gWT7'),
 (5,
  85.68329718004338,
  '‘วิษณุ’แย้มประตูยุบสภาเปิดอยู',
  'https://www.voicetv.co.th/read/HpsxqnPFI'),
 (0,
  85.48593350383632,
  'ถอดบทเรียนเอเปค สาวไส้ BCG ฟอ',
  'https://www.bbc.com/thai/thailand-63662204'),
 (2,
  83.0300465509945,
  '\'ประยุทธ์\' ลั่น "นายกฯ ไม่ใช่',
  'https://www.voicetv.co.th/read/NseVwZrF2'),
 (1,
  81.66345536287733,
  '‘พิธา’ มอง ‘ประยุทธ์’ เหมือนย',
  'https://www.bbc.com/thai/thailand-63566397'),
 (6,
  80.27923211169285,
  "รีวิว 'Collective': นักข่าวไม",
  'https://www.voicetv.co.th/read/bn95YMr0a')]

In [37]:
webpage = pages[4]
if webpage in websites:
    print("already scanned! choose another!")
websites.add(webpage)

already scanned! choose another!


In [14]:

with open(path+'viewed_websites_th.txt',"w") as outfile:
    for line in websites:
        outfile.write(line+'\n')      


In [38]:
#display() #percent_threshold=0.95
art=filter_text(webpage,print_word_lvl=True)

word-level % known = 89.43364327979712


In [39]:
with open(path+'unknown_thai_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub("[\n\'\[\]]","",new_words)
    new_words = new_words.split(',')
    new_words = [line.strip() for line in new_words] #update regex
#new_words
print(len(new_words))
#known_manual = get_known(new_words)

88


In [40]:
import mitosheet
unk_df = pd.DataFrame(new_words)
unk_df.columns = ['word']
unk_df['status'] = pd.Series(['' for word in new_words])

In [41]:
mitosheet.sheet(unk_df, analysis_to_replay="id-zdmuiplscs")

MitoWidget(analysis_data_json='{"analysisName": "id-zdmuiplscs", "analysisToReplay": null, "code": [], "stepSu…

In [42]:
add_ints = [0, 2, 5, 8, 13, 17, 19, 23, 32, 35, 39, 42, 45, 48, 49, 51, 52, 54, 57, 58, 60, 66, 67, 68, 75, 79, 82, 85]

In [43]:
for i in add_ints:
    # Set a cell value in status
    unk_df.at[i, 'status'] = 'k'

In [44]:
translations = []
for i in range(0,len(unk_df)):
    try:
        translations.append(ts.google(unk_df.word.iloc[i]))
    except:
        translations.append('')
unk_df['translations']=translations


In [45]:
mitosheet.sheet(unk_df, analysis_to_replay="id-eeiwaalmsp")

MitoWidget(analysis_data_json='{"analysisName": "id-eeiwaalmsp", "analysisToReplay": null, "code": [], "stepSu…

In [46]:
add_words = list(unk_df.word[unk_df['status']=='k'])
print(len(add_words))
for word in add_words:
    vocab['white_listed'].add(word)

28


In [56]:
#manually remove from white
remove_from_white = []

for word in remove_from_white:
    if word in vocab['white_listed']:
        vocab['white_listed'].remove(word)

In [55]:
#manually add to white
add_to_white = []

for word in add_to_white:
    vocab['white_listed'].add(word)

In [47]:
#write to .json formats
df = vocab
df['white_listed'] = list(df['white_listed'])
df['black_listed'] = list(df['black_listed'])
with open(path+source_file, "w") as outfile:
    json.dump(df,outfile)
print(len(df['white_listed']))

2192


In [48]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

2192


In [49]:
print(webpage)
filter_text(webpage,print_word_lvl=True)

https://www.voicetv.co.th/read/0kUrVxJCm
word-level % known = 92.73034657650042


,knowns,unknowns,unk_words
0,POLITICS ECONOMICS WORLD ENTERTAINMENT WELL-BEING LOCAL NEWS BLOG VOICE PLAZA TV PROGRAMS LIVE POLITICS ECONOMICS WORLD ENTERTAINMENT WELL-BEING LOCAL NEWS BLOG VOICE PLAZA TV PROGRAMS ABOUT FAQ CONTACT TERM OF USE SCHEDULE ไม่พบผลการค้นหา,...,
1,...,"คุณกำลังอ่าน : 'ประยุทธ์' ลั่น ""นายกฯ ไม่ใช่คนใจร้าย"" เผยไหว้พระขอพรทุกวันให้ประเทศเดินหน้า Share Tweet Share การเมือง 'ประยุทธ์' ลั่น ""นายกฯ ไม่ใช่คนใจร้าย"" เผยไหว้พระขอพรทุกวันให้ประเทศเดินหน้า Nov 25, 2022 (","""ลั่น"", ""ไหว้พระ"", ""ขอพร"", ""ลั่น"", ""ไหว้พระ"", ""ขอพร"""
2,"Last update Nov 25, 2022 11:28 ) นายกรัฐมนตรี",...,
3,...,มอบนโยบายเลือกตั้งสมาชิกสภาเกษตรกรฯ ขอให้สุจริต บอกไทยไม่มีภูเขาไฟ แต่ระเบิดกันเอง,"""มอบ"", ""เกษตรกร"", ""สุจริต"""
4,แนะควรภูมิใจที่อยู่แผ่นดินนี้,...,
...,...,...,...
68,ไม่มีสำเร็จสักอัน,...,
69,เชื่อผม ผมอยู่มาหลายปี,...,
70,ผมรู้ว่ามันมีปัญหาเยอะพอสมควรกับความไม่เข้าใจ ก็ขอให้ทุกคนทำความเข้าใจ,...,
71,...,เราต้องร่วมมือและจับมือกันเดินไปข้างหน้าพร้อมกัน,"""ร่วมมือ"""


##END

In [147]:
#Upload from download
with open(path+'known_thai_list.txt',"r") as input_file:
    new_words = input_file.read().split(',')
    print("daily catch in word count:")
    print(new_words)
for line in new_words:   
    vocab['white_listed'].add(line.replace("'","").strip())


daily catch in word count:
['เชิญ', 'ยิ้ม', 'ยินดี', 'รอย', 'เอกสาร', 'สนิท', 'กระดาษ', 'คอ', 'โรงพยาบาล', 'หิน', 'จด', 'ปืน', 'โต๊ะ', 'เสื้อผ้า', 'เลือด', 'สิทธิ์', 'นักเขียน', 'สนุกสนาน', 'หัวเราะ', 'หู', 'ยิง', 'สวยงาม', 'ล่า', 'แขก', 'ดอกไม้', 'ที่นั่ง', 'เสือ', 'ปกป้อง', 'รับประทาน', 'ไอ', 'สนุก', 'ลูกชาย', 'สหรัฐ', 'นิ้ว', 'ไอ้', 'รถไฟ', 'เล็กน้อย', 'โปรด', 'คะแนน', 'กรุณา', 'โรงแรม', 'จดหมาย', 'ขับ', 'ดื่ม', 'หอม', 'ระมัดระวัง', 'ฤดู', 'ฟัน', 'เท้า', 'หล่อน', 'วางแผน', 'ลับ', 'ลอย', 'ยกเว้น', 'ลูกค้า', 'ริม', 'ม้า', 'คอมพิวเตอร์']


In [154]:

wyw_text = 'สวัสดีครับน้อง'
print(ts.google(wyw_text))
#print(ts._google.language_map)
#print(ts.google(wyw_text))

Hello Nong


In [227]:
#Get after when adding in new words
print("today's catch % of corpus:")
str(100*(percent-prev_percent))+"%"

today's catch % of corpus:


'1.200344766395417%'

In [ ]:
#output newly knowns
with open(path+"new_known_thai_lines.txt","w") as outfile:
    for line in list(set(knowns).difference(prev_knowns)):
        outfile.write(line+'\n')


In [105]:
print("Sanity check. Current vocab size:")
len(vocab['white_listed'])

Sanity check. Current vocab size:


1462

In [117]:
"""
to do:

"""

In [7]:
##INPUT-OUTPUT
#vocab list:
with open("/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/thai_whitelisted.csv","r") as infile:
    whitelisted_lemmas = infile.read()


##VARIABLES
vocab_all = set()


for line in whitelisted_lemmas.split('\n'):
    if len(line) > 0:
        
        vocab = line.strip()
        vocab_all.add(vocab)
        
len(whitelisted_lemmas.split('\n'))            
#pp.pprint(vocab_all)

999

In [8]:
vocab = {}
vocab['white_listed'] = list(vocab_all)
vocab['black_listed'] = []